# 0. Correlation Analysis

This notebook performs an $f$-$k$ analysis and filtering, as well as phase velocity dispersion estimation.

# 0. Packages and setup

Import the necessary Python packages and add a few lines to embellish figures.

In [ ]:
import obspy
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 50})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

# 1. General input

Basic input, including the file to be read and the average spacing between channels.

In [ ]:
# Input file.
input_file='/Users/andreas/Desktop/Andreas_npy_files/bern_5_50Hz_corr_stacked_1s.npy'
# Scale for plotting.
scale=2
# Receiver spacing.
dx=1.9411132335662842

# 2. Data reading and plotting

We first read the correlation data and plot them.

In [ ]:
cct=np.load(input_file)

- (Average) distance between channels = 1.9411132335662842 m
- Sampling rate = 200 Hz
- 304 channels (950 - 1306) (number gap due to cutting out chunks)
- 1 hour, n_stack = 12, 002115 - 012115 UTC (01:21:15 - 02:21:15 local time)
- Filtered 5 - 50 Hz

In [ ]:
print(cct.shape)
nt=cct.shape[1]
nx=cct.shape[0]-1

In [ ]:
plt.figure(figsize=(20,20))

for i,j in enumerate(cct):
    
    if i % 100 == 0:
        print(f"At {i} of {np.shape(cct)[0]}")
    
    if i == 0:
        t=j
        dt=t[1]-t[0]
    else:
        data = j
        
        data /= np.max(np.abs(data))        
        dist_var = (i-1)*dx

        plt.plot(t,(scale*data)+dist_var,'k-', alpha = 0.4)
        plt.fill_between(t,(scale*data)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('Lag [s]')
plt.ylabel('Distance [m]')
plt.title('cross correlations',pad=20)
plt.grid()
plt.show()

# 3. Frequency-wavenumber domain

Before any $f$-$k$ filtering and dispersion analysis, we transform the time-domain data into the frequency-wavenumber domain.

## 3.1. Extract causal or acausal parts

We choose either the causal or the acausal part of the correlations for our analysis.

In [ ]:
# Pick causal or acausal part.
causal=False

In [ ]:
if causal:
    cct_cropped=cct[1:,int((nt-1)/2):nt]
else:
    cct_cropped=cct[1:,int((nt-1)/2)+1:0:-1]
    
t_cropped=np.linspace(0.0,(nt-1)*dt/2.0,int((nt-1)/2)+1)

In [ ]:
plt.figure(figsize=(10,20))

for i in range(nx):
    
    data=cct_cropped[i,:]
    data/=np.max(np.abs(data))        
    dist_var=(i-1)*dx
    
    plt.plot(t_cropped,(scale*data)+dist_var,'k-', alpha = 0.4) 
    plt.fill_between(t_cropped,(scale*data)+dist_var,y2=np.ones(np.shape(t_cropped))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.grid()
plt.show()

## 3.2. Compute and plot frequency-domain representation

We apply a 2D Fast Fourier Transform and the roll the indices in order to bring it into the physical frequency domain.

In [ ]:
# Reduced number of time samples.
nt_cropped=len(t_cropped)
# 2D Fourier transform.
ccf=np.fft.fft2(cct_cropped)
# Roll in order to plot in physical f-k domain.
ccfr=np.roll(np.roll(ccf,int((nt_cropped-1)/2),axis=1),int((nx-1)/2),axis=0)
# Make frequency and wavenumber axes.
f=np.linspace(-0.5/dt,0.5/dt,nt_cropped)
k=np.linspace(-np.pi/dx,np.pi/dx,nx)
ff,kk=np.meshgrid(f,k)

In [ ]:
# Plot discrete 2D Fourier transform
plt.subplots(1, figsize=(25,25))
plt.pcolor(np.abs(ccf),cmap='Greys')
plt.xlabel('frequency index')
plt.ylabel('space index')
plt.title('2D Fourier transform',pad=25)
plt.grid()
plt.show()

# Plot f-k domain amplitude spectrum.
plt.subplots(1, figsize=(25,25))
plt.pcolor(ff,kk,np.abs(ccfr),cmap='Greys')
plt.xlim(-50.0,50.0)
plt.ylim(-0.5,0.5)
plt.xlabel('f [Hz]')
plt.ylabel('k [1/m]')
plt.title('f-k power spectrum',pad=25)
plt.grid()
plt.show()

# 4. $f$-$k$ filtering

For $f$-$k$ filtering we define a mask that cuts out part of the $f$-$k$ spectrum.

## 4.1. Input

Set parameters that define the frequency-wavenumber mask.

In [ ]:
# Minimum and maximum absolute phase velocities [m/s].
c_min=300.0
c_max=2000.0
# Minimum and maximum frequencies [Hz].
freqmin=0.0
freqmax=50.0

## 4.2. $f$-$k$ mask

Build the $f$-$k$ mask for both the forward- and backward-propagating (reflected) part of the correlation wavefield.

In [ ]:
# Mask for forward-propagating wavefield.
mask_fwd=np.ones(np.shape(ccfr),dtype='complex64')
# Mask for backward-propagating (reflected) wavefield.
mask_rev=np.ones(np.shape(ccfr),dtype='complex64')

# Apply the mask.
for i in range(nx):
    for j in range(nt_cropped):
        
        # Compute phase velocity.
        if np.abs(k[i]):
            c=2.0*np.pi*f[j]/k[i]
        else:
            c=1.0e9
            
        # Maximum and minimum absolute phase velocities.
        if (np.abs(c)>c_max) or (np.abs(c)<c_min):
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
        # Forward or backward propagation.
        if c>0.0: mask_fwd[i,j]=0.0
        if c<0.0: mask_rev[i,j]=0.0
        # Bandpass filter.
        if np.abs(f[j])>freqmax: 
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
        if np.abs(f[j])<freqmin: 
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
            
            
# Smooth the mask.
dummy_fwd=mask_fwd.copy()
mask_fwd_smooth=mask_fwd.copy()
dummy_rev=mask_rev.copy()
mask_rev_smooth=mask_rev.copy()

for l in range(4):
    for i in np.arange(1,nx-1):
        for j in np.arange(1,nt_cropped-1):
            mask_rev_smooth[i,j]=(dummy_rev[i,j]+dummy_rev[i-1,j]+dummy_rev[i+1,j]+dummy_rev[i,j-1]+dummy_rev[i,j+1])/5.0
            mask_fwd_smooth[i,j]=(dummy_fwd[i,j]+dummy_fwd[i-1,j]+dummy_fwd[i+1,j]+dummy_fwd[i,j-1]+dummy_fwd[i,j+1])/5.0
    dummy_rev=mask_rev_smooth
    dummy_fwd=mask_fwd_smooth

In [ ]:
# Plot f-k domain masks.

plt.subplots(1, figsize=(25,25))
plt.pcolor(ff,kk,np.abs(mask_fwd_smooth),cmap='Greys')
plt.xlim(-1.1*freqmax,1.1*freqmax)
plt.ylim(-2.2*np.pi*freqmax/c_min,2.2*np.pi*freqmax/c_min)
plt.xlabel('f [Hz]')
plt.ylabel('k [1/m]')
plt.title('f-k mask (forward)',pad=25)
plt.colorbar()
plt.show()

plt.subplots(1, figsize=(25,25))
plt.pcolor(ff,kk,np.abs(mask_rev_smooth),cmap='Greys')
plt.xlim(-1.1*freqmax,1.1*freqmax)
plt.ylim(-2.2*np.pi*freqmax/c_min,2.2*np.pi*freqmax/c_min)
plt.xlabel('f [Hz]')
plt.ylabel('k [1/m]')
plt.title('f-k mask (reverse)',pad=25)
plt.colorbar()
plt.show()

In [ ]:
# Apply the f-k domain filter.
ccfr_fwd_filtered=ccfr*mask_fwd_smooth
ccfr_rev_filtered=ccfr*mask_rev_smooth

## 4.3. Return to the time domain

After application of the mask (filter) we roll the arrays back and then apply the 2D inverse transform.

In [ ]:
# Roll back.
ccf_fwd_filtered=np.roll(np.roll(ccfr_fwd_filtered,-int((nt_cropped-1)/2),axis=1),-int((nx-1)/2),axis=0)
ccf_rev_filtered=np.roll(np.roll(ccfr_rev_filtered,-int((nt_cropped-1)/2),axis=1),-int((nx-1)/2),axis=0)

# Plot to check.
#plt.subplots(1, figsize=(25,25))
#plt.pcolor(np.abs(ccf_fwd_filtered),cmap='Greys')
#plt.xlabel('frequency index')
#plt.ylabel('space index')
#plt.title('2D Fourier transform, filtered',pad=25)
#plt.show()

# Inverse 2D Fourier transform.
cct_fwd_filtered=np.real(np.fft.ifft2(ccf_fwd_filtered))
cct_rev_filtered=np.real(np.fft.ifft2(ccf_rev_filtered))

In [ ]:
# Plotting options.
plot_forward=True
plot_reverse=True

# Plot filtered version.
plt.figure(figsize=(10,20))

for i in range(nx):
    
    dist_var=(i-1)*dx
    
    if plot_forward:
        data_fwd=cct_fwd_filtered[i,:]
        data_fwd/=np.max(np.abs(cct_cropped[i,:]))  
        plt.plot(t_cropped,(scale*data_fwd)+dist_var,'k-', alpha = 0.4) 
        plt.fill_between(t_cropped,(scale*data_fwd)+dist_var,y2=np.ones(np.shape(t_cropped))*dist_var,where=(data_fwd+dist_var>=dist_var), interpolate=True,fc='b',alpha=0.8)
        
    if plot_reverse:
        data_rev=cct_rev_filtered[i,:]
        data_rev/=np.max(np.abs(cct_cropped[i,:]))   
        plt.plot(t_cropped,(scale*data_rev)+dist_var,'k-', alpha = 0.4) 
        plt.fill_between(t_cropped,(scale*data_rev)+dist_var,y2=np.ones(np.shape(t_cropped))*dist_var,where=(data_rev+dist_var>=dist_var), interpolate=True,fc='r',alpha=0.8)

plt.xlabel("Lag [s]")
plt.ylabel("Distance [m]")
plt.grid()
plt.show()

# 5. Phase velocity dispersion measurements

We estimate phase velocity dispersion by picking the maximum of the $k$-dependent spectrum.

## 5.1. Input

Choose the frequency band for the dispersion estimation, and the degree of spectral smoothing.

In [ ]:
# Minimum and maximum frequencies for dispersion measurements [Hz].
fd_min=5.0
fd_max=50.0
# Number of neighbourhood smoothing iterations.
n_smooth=2

In [ ]:
# Plot f-k domain amplitude spectrum.

plt.subplots(1, figsize=(25,25))
plt.pcolor(ff,kk,np.abs(ccfr_fwd_filtered),cmap='Greys')
plt.xlim(-1.1*freqmax,1.1*freqmax)
plt.ylim(-2.2*np.pi*freqmax/c_min,2.2*np.pi*freqmax/c_min)
plt.xlabel('f [Hz]')
plt.ylabel('k [1/m]')
plt.title('f-k power spectrum, filtered, forward',pad=25)
plt.grid()
plt.show()

plt.subplots(1, figsize=(25,25))
plt.pcolor(ff,kk,np.abs(ccfr_rev_filtered),cmap='Greys')
plt.xlim(-1.1*freqmax,1.1*freqmax)
plt.ylim(-2.2*np.pi*freqmax/c_min,2.2*np.pi*freqmax/c_min)
plt.xlabel('f [Hz]')
plt.ylabel('k [1/m]')
plt.title('f-k power spectrum, filtered, reverse',pad=25)
plt.grid()
plt.show()

In [ ]:
# Find start and end indices of frequency band.
idx_min=np.where(np.min(np.abs(f-fd_min))==np.abs(f-fd_min))[0][0]
idx_max=np.where(np.min(np.abs(f-fd_max))==np.abs(f-fd_max))[0][0]

In [ ]:
# Phase velocities of the froward and reverse parts.
c_fwd=np.zeros(nt_cropped)
c_rev=np.zeros(nt_cropped)

# March through frequency indices.
for idx in np.arange(idx_min,idx_max+1):
    
    # Make a smoother version of the k-dependent spectrum.
    amp_fwd=np.abs(ccfr_fwd_filtered[:,idx])
    amp_rev=np.abs(ccfr_rev_filtered[:,idx])
    for i in range(n_smooth): 
        amp_fwd[1:nt_cropped-1]=(2.0*amp_fwd[1:nt_cropped-1]+amp_fwd[0:nt_cropped-2]+amp_fwd[2:nt_cropped])/4.0
        amp_rev[1:nt_cropped-1]=(2.0*amp_rev[1:nt_cropped-1]+amp_rev[0:nt_cropped-2]+amp_rev[2:nt_cropped])/4.0
    
    # Compute phase velocity by picking the maximum.
    j_fwd=np.argmax(amp_fwd) 
    j_rev=np.argmax(amp_rev) 
    c_fwd[idx]=np.abs(2.0*np.pi*f[idx]/k[j_fwd])
    c_rev[idx]=np.abs(2.0*np.pi*f[idx]/k[j_rev])

In [ ]:
plt.subplots(1, figsize=(25,15))

plt.plot(f,c_fwd,'--b',linewidth=2)
plt.plot(f,c_rev,'--r',linewidth=2)
plt.plot(f,0.5*(c_rev+c_fwd),'--k',linewidth=6)
plt.plot([f[idx_min],f[idx_max]],[c_max,c_max],'--k',linewidth=2)
plt.plot([f[idx_min],f[idx_max]],[c_min,c_min],'--k',linewidth=2)

plt.xlim([f[idx_min],f[idx_max]])
plt.ylim([c_min-50.0,c_max+50.0])
plt.xlabel('frequency [Hz]',labelpad=15)
plt.ylabel('phase velocity [m/s]',labelpad=15)
plt.show()